In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df=pd.read_csv('anime.csv')

In [3]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
df.shape

(12294, 7)

In [5]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [6]:
# filling the numerical missing values only
df['rating']=df['rating'].fillna(0)

In [7]:
df['episodes']=df['episodes'].replace('Unknown',0)

In [8]:
df['episodes']=df['episodes'].astype(int)

In [9]:
df.isnull().sum()

anime_id     0
name         0
genre       62
type        25
episodes     0
rating       0
members      0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.shape

(12210, 7)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
new_genre=[]
for x in df['genre']:
    genre=''.join(x.split(','))
    new_genre.append(genre)
    

In [14]:
df['new_genre']=new_genre

In [15]:
tfidf=TfidfVectorizer()

In [16]:
genre_matrix=tfidf.fit_transform(df['new_genre'])

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scale=StandardScaler()

In [19]:
scaled_values=scale.fit_transform(df[['episodes','rating']])

In [20]:
scaled_values

array([[-0.23949859,  2.30692766],
       [ 1.11812483,  2.22217465],
       [ 0.83798032,  2.21446983],
       ...,
       [-0.17484985, -1.15253622],
       [-0.23949859, -1.07548803],
       [-0.23949859, -0.7056567 ]])

In [21]:
from scipy.sparse import hstack 

In [22]:
combined_features=hstack([genre_matrix,scaled_values])

In [23]:
combined_features

<12210x49 sparse matrix of type '<class 'numpy.float64'>'
	with 65988 stored elements in COOrdinate format>

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
cosine_sim=cosine_similarity(combined_features,combined_features)

In [26]:
np.fill_diagonal(cosine_sim,0)

In [27]:
cosine_df=pd.DataFrame(cosine_sim,index=df['anime_id'],columns=df['anime_id'])

In [37]:
def recommend(anime_id, n=5):
    
    sims = cosine_df.loc[anime_id]
    
    
    top_ids = sims.nlargest(n).index.tolist()
    

    return df[df['anime_id'].isin(top_ids)][['name']]


In [38]:
recommend(32281)

,name
15,Sen to Chihiro no Kamikakushi
60,Hotarubi no Mori e
159,Angel Beats!
208,Kokoro ga Sakebitagatterunda.
219,Yahari Ore no Seishun Love Comedy wa Machigatt...
